# Exploration.

In [213]:
from pymongo import MongoClient
import pandas as pd
import time
import importlib
import geopandas as gpd

import json
from cartoframes.viz import Map, Layer, popup_element

from getpass import getpass
import os
import requests

from dotenv import load_dotenv
import pymongo
import warnings
warnings.filterwarnings("ignore")
import data.exploration as ex
importlib.reload(ex)

<module 'data.exploration' from 'd:\\ironhack\\proyectos\\GeoSpatialData_proy3\\data\\exploration.py'>

## 1. Ubication.
### 1.1. Import from Mongo.

Todo el proceso para descargar la base de datos Empresas de Mongo.

In [2]:
client = MongoClient("localhost:27017")

In [3]:
client.list_database_names()

['Ironhack', 'admin', 'config', 'local']

In [4]:
db = client["Ironhack"]

collection for companies Mongo.

In [5]:
c = db.get_collection("companies")

collection for NY neighberhood.

In [329]:
nyneights = db.get_collection("nyneigh")

Explore the data base to get the information I want for the company.

In [6]:
# This was to look if the data base from Mongo was well downloaded.
    # c.find_one()
# This was to find with a specific condition of the ategory, and look for what i wanted to work on.
    # list(c.find({}, {"category_code":1}))

#### Will choose the music industry for my company.
Getting the music buissnes companies and start analizing them.

In [8]:
# list of all the companies with category of music in all the world.
    # list(c.find({"category_code":"music"}, {"name":1, "_id":0, "category_code":1}))

### 1.2. Ubication COUNTRY.

In [9]:
# list of the countries with the music company industry.
lstcountry = ex.music_countries ("music", 1)

In [10]:
#countries[0]
#countries[0]["offices"]
#countries[0]["offices"][0]["country_code"]

In [11]:
# change it to a dictionary
country = {"country": lstcountry}

In [172]:
# make it a data frame
dfcountry = ex.make_df (country)

We see that for much USA is the place for a music company, I wanted to see if I could go to another country that had some type of competition between the USA, but if the company wants to grow, need to be in the USA.

hacer otra columna del value count de los paises

In [14]:
dfcountry.sample(3)

,country
24,ESP
47,ITA
17,USA


### 1.3. Searching for the city

In [15]:
# Extracting from mongo list of cities chosing the category of the business.

In [16]:
lstcity = ex.music_cities ("music", 1)

Merge the 2 columns of country and city and new count column

In [17]:
location_df = ex.combine_lists (lstcity, lstcountry)

In [18]:
location_df.sample(3)

,cities,country,count
36,Atlanta,USA,2
46,,USA,1
22,Oslo,NOR,1


Clean the data frame, from dulicates and nulls and create a new column for counts of offices in each city.

In [19]:
location_clean = ex.clean_df_citycountry (location_df)
location_clean.sample(3)

,cities,country,count
1,New York,USA,6
35,Atlanta,USA,2
24,Madrid,ESP,1


Export the data frame of location into a CSV file.

In [20]:
ex.export_location (location_clean)

## 2. Searching for music competition.

Filtering from mongo name, latitude and longitud. to a dictionary.

In [178]:
music_ny = ex.music_NY ("music", "New York")

In [176]:
lat_music = ex.lat_ (music_ny)

[40.725435, 40.72948, 37.872222, 49.607295, 42.35888, None]

In [177]:
long_music = ex.long_ (music_ny)

Create DF of latitude and longitude.

In [52]:
df_music = ex.coord_df (lat_music, long_music)

In [53]:
musicdf = ex.musicdf_NY ("music", "New York")

In [54]:
music_df = ex.concat_axis1 (musicdf, df_music)

In [546]:
ex.export_NY_music (music_df)

## 3. Startup environment.

Filter information.

In [193]:
startups = ex.startups_cooord ("New York", 1000000)

In [183]:
startups_lat = ex.lat_ (startups)

In [184]:
startups_long = ex.long_ (startups)

Filter again the startups but without the latitud and logitude and turn it into a data frame.


In [195]:
start_names = ex.startups_name ("New York", 1000000)
start_names.sample(3)

,name,category_code,acquisitions
0,Facebook,social,"[{'price_amount': None}, {'price_amount': 4750..."
30,Sapient,enterprise,"[{'price_amount': 50000000}, {'price_amount': ..."
27,F5 Networks,other,"[{'price_amount': 210000000}, {'price_amount':..."


Make a dictionary with all values to create a data base.

In [286]:
#     ex.price_amount (city, amount)

In [186]:
df = ex.coord_df (startups_lat, startups_long)

In [187]:
df.sample(3)

,latitud,longitude
29,41.889853,-87.635543
31,NaN,NaN
1,40.757725,-73.986011


Concat 2 data frames.

In [188]:
startups = ex.concat_axis1 (start_names, df)

In [189]:
startups.sample()

,name,category_code,latitud,longitude
30,Sapient,enterprise,NaN,NaN


In [43]:
with open ("startupsNYjson.json", "w") as f:
    json.dump(startups, f)

In [564]:
ex.export_NY_startups (startups)

## 4. Foursquare.

In [62]:
token = getpass()

### 4.1. Starbucks.

In [349]:
square_starbucks = ex.foursquare ("starbucks", 13032)

In [350]:
len(square_starbucks)

50

In [262]:
single = ex.single_coord_name (square_starbucks[0])

In [263]:
dict_startbucks = ex.iterate_all (square_starbucks)

In [384]:
starbucks_df = ex.make_df (dict_startbucks)

Create a new collection and add/ insert all the rows in the data frame.

In [266]:
for index, row in starbucks_df.iterrows():
    db["ny_starbucks"].insert_one(dict(row))

In [296]:
mongo_starbucks = db.get_collection("ny_starbucks")
mongo_starbucks

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack'), 'ny_starbucks')

In [297]:
mongo_starbucks.find_one()

{'_id': ObjectId('63ea36db2549203038bc6b92'),
 'name': 'Starbucks',
 'latitude': 40.739165,
 'longitude': -74.002468}

In [353]:
coords_starbucks = list(mongo_starbucks.find({},{"_id":0, "latitude":1, "longitude":1} ))
len(coords_starbucks)

50

In [354]:
latitude = []
for i in range(len(coords_starbucks)):
    latitude.append(coords_starbucks[i]["latitude"])
len(latitude)

50

In [363]:
longitude = []
for i in range(len(coords_starbucks)):
    longitude.append(coords_starbucks[i]["longitude"])


In [365]:
d = []
for i in range(len(latitude)):
    d.append({"type": "Point", "coordinates": [longitude[i], latitude[i]]})
len(d)


50

In [370]:
w = []
for i in d:
    x = list(nyneights.find(
    {"geometry":
        {"$geoIntersects": 
        {"$geometry": i}}}, projection = {"name": 1, "_id":0}))
    w.append(x)

In [378]:
starbucks_neigh = [j for i in w for j in i]


In [387]:
starbucks_neigh_df = ex.make_df (starbucks_neigh)

crear nueva columna con value counts de la cantidad de starbucks en cada neightberhood

In [393]:
starbucks_neigh_df["count"] = starbucks_neigh_df.groupby(["name"])["name"].transform("count")

eliminar duplicate

In [394]:
starbucks_neigh_df.drop_duplicates(['name','count'],keep= 'last')

,name,count
21,West Village,9
28,Gramercy,3
42,Stuyvesant Town-Cooper Village,1
44,East Village,2
45,Midtown-Midtown South,13
47,Murray Hill-Kips Bay,3
48,SoHo-TriBeCa-Civic Center-Little Italy,5
49,Hudson Yards-Chelsea-Flatiron-Union Square,14


Export data frame to csv file.

In [267]:
ex.export_NY_starbucks (starbucks_df)

### 4.2. Schools.

In [268]:
square_schools = ex.foursquare ("schools", 12009)

In [269]:
single_schools = ex.single_coord_name (square_schools[0])

In [270]:
dict_schools = ex.iterate_all (square_schools)

In [271]:
school_df = ex.make_df (dict_schools)

Create a new collection and add/ insert all the rows in the data frame.

In [272]:
for index, row in school_df.iterrows():
    db["ny_schools"].insert_one(dict(row))

Export data frame to csv file.

In [273]:
ex.export_NY_schools (school_df)

### 4.3. Music scenario.

### 4.3.1. Bars.

In [274]:
square_bars = ex.foursquare ("bars", 13003)

In [275]:
single_bars = ex.single_coord_name (square_bars[0])

In [276]:
dict_bars = ex.iterate_all (square_bars)

In [277]:
bars_df = ex.make_df (dict_bars)

Create a new collection and add/ insert all the rows in the data frame.

In [278]:
for index, row in bars_df.iterrows():
    db["ny_bars"].insert_one(dict(row))

Export data frame to csv file.

In [279]:
ex.export_NY_bars (school_df)

### 4.3.2. Street concerts.

In [280]:
square_concerts = ex.foursquare ("concerts", 10000)

In [281]:
single_concerts = ex.single_coord_name (square_concerts[0])

In [282]:
dict_concerts = ex.iterate_all (square_concerts)

In [283]:
concerts_df = ex.make_df (dict_concerts)

Create a new collection and add/ insert all the rows in the data frame.

In [284]:
for index, row in concerts_df.iterrows():
    db["ny_concerts"].insert_one(dict(row))

Export Data frame as csv file.

In [285]:
ex.export_NY_concerts (school_df)

In [ ]:
# to create a new collection in mongo.
# db["ny_schools"].insert_one({"algo":355})